In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



In [7]:
FD_OUT=${FD_RES}/peak/cradle
FN_OUT=cradle_active_peaks_resize.bed
head ${FD_OUT}/${FN_OUT}

chr1	778232	779232	chr1:778588-778877:DMSO
chr1	818441	819441	chr1:818351-819531:Dex
chr1	826644	827644	chr1:826715-827573:DMSO
chr1	851117	852117	chr1:851268-851966:DMSO
chr1	856205	857205	chr1:856338-857072:Dex
chr1	873413	874413	chr1:873788-874038:DMSO
chr1	873738	874738	chr1:874088-874388:Dex
chr1	874213	875213	chr1:874538-874888:Dex
chr1	905025	906025	chr1:905113-905938:Dex
chr1	936253	937253	chr1:936528-936978:DMSO


In [3]:
ls ${FD_RES}/data/

Input       Input2      Input3_20x  Input5      TFX2_DMSO  TFX4_Dex   TFX5_DMSO
Input1      Input2_20x  Input4      Input5_20x  TFX3_Dex   TFX4_DMSO  TFX_Dex
Input1_20x  Input3      Input4_20x  TFX2_Dex    TFX3_DMSO  TFX5_Dex   TFX_DMSO


In [5]:
ls ${FD_RES}/data/Input1_20x

chr10.bed  chr14.bed  chr18.bed  chr21.bed  chr4.bed  chr8.bed  target_PER1.bed
chr11.bed  chr15.bed  chr19.bed  chr22.bed  chr5.bed  chr9.bed
chr12.bed  chr16.bed  chr1.bed   chr2.bed   chr6.bed  chrX.bed
chr13.bed  chr17.bed  chr20.bed  chr3.bed   chr7.bed  chrY.bed


In [6]:
FD_BED=${FD_RES}/data/Input1_20x
FN_BED=chr1.bed
head ${FD_BED}/${FN_BED}

chr1	13802	14757
chr1	13802	14759
chr1	13802	14759
chr1	13868	14793
chr1	13868	14793
chr1	14376	15250
chr1	14424	15516
chr1	14448	15495
chr1	14449	15491
chr1	14713	15705


In [17]:
FD_BEDS=($(ls -d ${FD_RES}/data/{Input?*,TFX?_DMSO,TFX?_Dex}/))
#FD_BEDS=($(ls -d ${FD_RES}/count_fragment/Input?_20x/))

### Loop folders
for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    echo "SAMPLE: ${SAMPLE}; FD_BED: ${FD_BED}"
done

SAMPLE: Input1; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1/
SAMPLE: Input1_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
SAMPLE: Input2; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2/
SAMPLE: Input2_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
SAMPLE: Input3; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3/
SAMPLE: Input3_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
SAMPLE: Input4; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4/
SAMPLE: Input4_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
SAMPLE: Input5; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5/
SAMPLE: Input5_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/
SAMPLE: TFX2_Dex; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_Dex/
SAMPLE: TF

In [22]:
### set env
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --array 0-22 \
    --mem 20G \
    -o ${FD_LOG}/peak_fragment_count_chrom.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=${CHROM}
#TARGET=chr17

### set input and output
FD_BEDS=($(ls -d ${FD_RES}/data/{Input?*,TFX?_DMSO,TFX?_Dex}/))
#FD_BEDS=($(ls -d ${FD_RES}/data/Input?/))
FN_BED=${TARGET}.bed

### peak annotation
FD_PEAK=${FD_RES}/peak/cradle
FN_PEAK=cradle_active_peaks_resize.bed

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### loop through each sample and count fragments on each peak
echo "RUN: loop through samples"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/peak/peak_count_cradle/${SAMPLE}
    FN_INT=${TARGET}_intersect.bed.gz
    FN_CNT=${TARGET}_count.bed.gz
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_PEAK}/${FN_PEAK}
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_PEAK}/${FN_PEAK}
    cat ${FD_PEAK}/${FN_PEAK} | head -n 3 
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    cat ${FD_BED}/${FN_BED} | head -n 3 
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### intersect the peak and fragments
    bedtools intersect -a ${FD_PEAK}/${FN_PEAK} -b ${FD_BED}/${FN_BED} -wo |\
        gzip -c > ${FD_OUT}/${FN_INT}
    
    ### count the overlapped fragments for each peak
    zcat ${FD_OUT}/${FN_INT} |\
        cut -f1,2,3,4 |\
        sort    |\
        uniq -c |\
        awk '{$(NF+1)=$1;$1=""}1'   |\
        sed -e 's/^[[:space:]]*//'  |\
        sed -e 's/[[:space:]]/\t/g' |\
        sort -k1,1 -k2,2n |\
        gzip -c > ${FD_OUT}/${FN_CNT}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_INT}
    zcat ${FD_OUT}/${FN_INT} | head -n 10
    echo
    echo ${FD_OUT}/${FN_CNT}
    zcat ${FD_OUT}/${FN_CNT} | head -n 3
    echo
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26869103


In [23]:
tail ${FD_LOG}/peak_fragment_count_chrom.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/peak_fragment_count_chrom.0.txt <==
chr1	778232	779232	chr1:778588-778877:DMSO	chr1	777866	778878	646
chr1	778232	779232	chr1:778588-778877:DMSO	chr1	777866	778876	644

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/peak/peak_count_cradle/TFX5_DMSO/chr1_count.bed.gz
chr1	778232	779232	chr1:778588-778877:DMSO	44
chr1	818441	819441	chr1:818351-819531:Dex	31
chr1	826644	827644	chr1:826715-827573:DMSO	74

Done!
Run Time: 8 minutes and 51 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/peak_fragment_count_chrom.10.txt <==
chr11	201437	202437	chr11:201525-202350:Dex	chr11	200734	201733	296
chr11	201437	202437	chr11:201525-202350:Dex	chr11	200735	201731	294

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/peak/peak_count_cradle/TFX5_DMSO/chr11_count.bed.gz
chr11	201437	202437	chr11:201525-202350:Dex	28
chr11	207430	208430	chr11:206860-209000:DMSO	492
chr11	207612	208612	chr11:207425-208800:Dex	491

Done!
Run Time: 5